In [ ]:
!sudo apt-get install fluidsynth
!pip install pyfluidsynth
!pip install pretty_midi

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following packages were automatically installed and are no longer required:
  cuda-command-line-tools-10-0 cuda-command-line-tools-10-1
  cuda-command-line-tools-11-0 cuda-compiler-10-0 cuda-compiler-10-1
  cuda-compiler-11-0 cuda-cuobjdump-10-0 cuda-cuobjdump-10-1
  cuda-cuobjdump-11-0 cuda-cupti-10-0 cuda-cupti-10-1 cuda-cupti-11-0
  cuda-cupti-dev-11-0 cuda-documentation-10-0 cuda-documentation-10-1
  cuda-documentation-11-0 cuda-documentation-11-1 cuda-gdb-10-0 cuda-gdb-10-1
  cuda-gdb-11-0 cuda-gpu-library-advisor-10-0 cuda-gpu-library-advisor-10-1
  cuda-libraries-10-0 cuda-libraries-10-1 cuda-libraries-11-0
  cuda-memcheck-10-0 cuda-memcheck-10-1 cuda-memcheck-11-0 cuda-nsight-10-0
  cuda-nsight-10-1 cuda-nsight-11-0 cuda-nsight-11-1 cuda-nsight-compute-10-0
  cuda-nsight-compute-10-1 cuda-nsight-compute-11-0 cuda-nsight-compute-11-1
  cuda-nsight-systems-10-1 cuda-nsight-systems-

In [ ]:
import fluidsynth
import glob
import numpy as np
import pathlib
import pretty_midi
import tensorflow as tf
from IPython import display
from music21 import converter, instrument, note, chord, stream

from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import LSTM
from keras.layers import Activation
from keras.layers import BatchNormalization as BatchNorm
from keras.utils import np_utils
from keras.callbacks import ModelCheckpoint

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
data_dir = pathlib.Path('/content/drive/MyDrive/tsm_dataset')

In [ ]:
filenames = glob.glob(str(data_dir/'*.mid*'))
print('Number of files:', len(filenames))

Number of files: 62


In [ ]:
sample_file = filenames[np.random.randint(0,len(filenames))]
pm = pretty_midi.PrettyMIDI(sample_file)
print(sample_file)

/content/drive/MyDrive/tsm_dataset/meyan 5.mid


In [ ]:
pm = pretty_midi.PrettyMIDI(sample_file)

In [ ]:
def display_audio(pm: pretty_midi.PrettyMIDI, seconds=30):
  waveform = pm.fluidsynth(fs=16000)
  # Take a sample of the generated waveform to mitigate kernel resets
  waveform_short = waveform[:seconds*16000]
  return display.Audio(waveform_short, rate=16000)
display_audio(pm)

In [ ]:
notes = []

for file in glob.glob("/content/drive/MyDrive/tsm_dataset/*.mid"):
    midi = converter.parse(file)

    print("Parsing %s" % file)

    notes_to_parse = None

    try: # file has instrument parts
        s2 = instrument.partitionByInstrument(midi)
        notes_to_parse = s2.parts[0].recurse() 
    except: # file has notes in a flat structure
        notes_to_parse = midi.flat.notes

    for element in notes_to_parse:
        if isinstance(element, note.Note):
            notes.append(str(element.pitch))
        elif isinstance(element, chord.Chord):
            notes.append('.'.join(str(n) for n in element.normalOrder))

n_vocab = len(set(notes))

Parsing /content/drive/MyDrive/tsm_dataset/aranağme 1.2 (1).mid
Parsing /content/drive/MyDrive/tsm_dataset/aranağme 1.2.mid
Parsing /content/drive/MyDrive/tsm_dataset/aranağme 2.mid
Parsing /content/drive/MyDrive/tsm_dataset/aranağme 3.mid
Parsing /content/drive/MyDrive/tsm_dataset/aranağme 7.mid
Parsing /content/drive/MyDrive/tsm_dataset/aranağme 8.mid
Parsing /content/drive/MyDrive/tsm_dataset/aranağme 10.mid
Parsing /content/drive/MyDrive/tsm_dataset/aranağme 12.mid
Parsing /content/drive/MyDrive/tsm_dataset/zemin 5.mid
Parsing /content/drive/MyDrive/tsm_dataset/zemin 6.mid
Parsing /content/drive/MyDrive/tsm_dataset/zemin 7.mid
Parsing /content/drive/MyDrive/tsm_dataset/zemin 8.mid
Parsing /content/drive/MyDrive/tsm_dataset/zemin 9.mid
Parsing /content/drive/MyDrive/tsm_dataset/zemin 10.mid
Parsing /content/drive/MyDrive/tsm_dataset/zemin 11.mid
Parsing /content/drive/MyDrive/tsm_dataset/zemin 12.mid
Parsing /content/drive/MyDrive/tsm_dataset/zemin 13.mid
Parsing /content/dr

In [ ]:
sequence_length = 100

# get all pitch names
pitchnames = sorted(set(item for item in notes))

# create a dictionary to map pitches to integers
note_to_int = dict((note, number) for number, note in enumerate(pitchnames))

network_input = []
network_output = []

# create input sequences and the corresponding outputs
for i in range(0, len(notes) - sequence_length, 1):
  sequence_in = notes[i:i + sequence_length]
  sequence_out = notes[i + sequence_length]
  network_input.append([note_to_int[char] for char in sequence_in])
  network_output.append(note_to_int[sequence_out])

n_patterns = len(network_input)

# reshape the input into a format compatible with LSTM layers
network_input = np.reshape(network_input, (n_patterns, sequence_length, 1))
# normalize input
network_input = network_input / float(n_vocab)

network_output = np_utils.to_categorical(network_output)

In [ ]:
def create_network(network_input, n_vocab):
    """ create the structure of the neural network """
    model = Sequential()
    model.add(LSTM(
        512,
        input_shape=(network_input.shape[1], network_input.shape[2]),
        recurrent_dropout=0.3,
        return_sequences=True
    ))
    model.add(LSTM(512, return_sequences=True, recurrent_dropout=0.3,))
    model.add(LSTM(512))
    model.add(BatchNorm())
    model.add(Dropout(0.3))
    model.add(Dense(256))
    model.add(Activation('relu'))
    model.add(BatchNorm())
    model.add(Dropout(0.3))
    model.add(Dense(n_vocab))
    model.add(Activation('softmax'))
    model.compile(loss='categorical_crossentropy', optimizer='rmsprop')

    return model

In [ ]:
# model = create_network(network_input, n_vocab)
# filepath = "weights-improvement-{epoch:02d}-{loss:.4f}-bigger.hdf5"
# checkpoint = ModelCheckpoint(
#     filepath,
#     monitor='loss',
#     verbose=0,
#     save_best_only=True,
#     mode='min'
# )
# callbacks_list = [checkpoint]

# model.fit(network_input, network_output, epochs=100, batch_size=64, callbacks=callbacks_list)


Epoch 1/100
21/21 [==============================] - 37s 1s/step - loss: 3.9174
Epoch 2/100
21/21 [==============================] - 27s 1s/step - loss: 3.6567
Epoch 3/100
21/21 [==============================] - 26s 1s/step - loss: 3.3861
Epoch 4/100
21/21 [==============================] - 27s 1s/step - loss: 3.2933
Epoch 5/100
21/21 [==============================] - 27s 1s/step - loss: 3.1682
Epoch 6/100
21/21 [==============================] - 26s 1s/step - loss: 3.0157
Epoch 7/100
21/21 [==============================] - 26s 1s/step - loss: 2.8752
Epoch 8/100
21/21 [==============================] - 26s 1s/step - loss: 2.8063
Epoch 9/100
21/21 [==============================] - 26s 1s/step - loss: 2.7184
Epoch 10/100
21/21 [==============================] - 26s 1s/step - loss: 2.5802
Epoch 11/100
21/21 [==============================] - 26s 1s/step - loss: 2.5530
Epoch 12/100
21/21 [==============================] - 26s 1s/step - loss: 2.4999
Epoch 13/100
21/21 [=================

In [ ]:
model.load_weights('/content/weights-improvement-99-0.8139-bigger.hdf5')
# model = create_network(normalized_input, n_vocab)

In [ ]:
pitchnames = sorted(set(item for item in notes))

note_to_int = dict((note, number) for number, note in enumerate(pitchnames))

sequence_length = 100
network_input = []
output = []
for i in range(0, len(notes) - sequence_length, 1):
    sequence_in = notes[i:i + sequence_length]
    sequence_out = notes[i + sequence_length]
    network_input.append([note_to_int[char] for char in sequence_in])
    output.append(note_to_int[sequence_out])

n_patterns = len(network_input)

# reshape the input into a format compatible with LSTM layers
normalized_input = np.reshape(network_input, (n_patterns, sequence_length, 1))
# normalize input
normalized_input = normalized_input / float(n_vocab)

In [ ]:
start = np.random.randint(0, len(network_input)-1)
int_to_note = dict((number, note) for number, note in enumerate(pitchnames))
pattern = network_input[start]
prediction_output = []
# generate 500 notes
for note_index in range(100):
    prediction_input = np.reshape(pattern, (1, len(pattern), 1))
    prediction_input = prediction_input / float(n_vocab)
    prediction = model.predict(prediction_input, verbose=0)
    index = np.argmax(prediction)
    result = int_to_note[index]
    prediction_output.append(result)
    pattern.append(index)
    pattern = pattern[1:len(pattern)]

In [ ]:
offset = 0
output_notes = []

# create note and chord objects based on the values generated by the model
for pattern in prediction_output:
    # pattern is a chord
    if ('.' in pattern) or pattern.isdigit():
        notes_in_chord = pattern.split('.')
        notes = []
        for current_note in notes_in_chord:
            new_note = note.Note(int(current_note))
            new_note.storedInstrument = instrument.Piano()
            notes.append(new_note)
        new_chord = chord.Chord(notes)
        new_chord.offset = offset
        output_notes.append(new_chord)
    # pattern is a note
    else:
        new_note = note.Note(pattern)
        new_note.offset = offset
        new_note.storedInstrument = instrument.Piano()
        output_notes.append(new_note)

    # increase offset each iteration so that notes do not stack
    offset += 0.5


In [ ]:
midi_stream = stream.Stream(output_notes)

midi_stream.write('midi', fp='/content/drive/MyDrive/test_output.mid')

'/content/drive/MyDrive/test_output.mid'